### Project Overview ###
This project is a simple binary classdification project. The final goal of this project is to use various external variables, such as age of the customer, balance of customer's account, and how many campaigns customer saw in the past, to determine **whether a customer would subscribe to a term deposit offered by the bank**.   
   
In the first part, I will do EDA to quickly understand how the data look like. At the same time, I will clean the data and finish preprocessings to get the data ready for further modeling. The data source and detail explanation of the columns could be found from this kaggle page. (https://www.kaggle.com/datasets/thedevastator/bank-term-deposit-predictions)

In [5]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split,KFold,RandomizedSearchCV,GridSearchCV
from sklearn import metrics,svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import lightgbm as lgb
from matplotlib.pyplot import figure
from sklearn.metrics import recall_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from tensorflow import keras
from keras.utils import np_utils
from keras import layers
from keras_tuner.tuners import RandomSearch
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import os

In [44]:
# Load the train data
train = pd.read_csv("/Users/haochunniu/Desktop/Kaggle Compatition/Bank Term Deposit Predictions NN ML Nested Random Search/data/train.csv")
n_row,n_col = train.shape[0],train.shape[1]-1
print("There are totally {} rows in train data with {} external variables".format(n_row,n_col))

# Load the test data
test = pd.read_csv("/Users/haochunniu/Desktop/Kaggle Compatition/Bank Term Deposit Predictions NN ML Nested Random Search/data/test.csv")
n_row,n_col = test.shape[0],test.shape[1]-1
print("There are totally {} rows in test data with {} external variables".format(n_row,n_col))

There are totally 45211 rows in train data with 16 external variables
There are totally 4521 rows in test data with 16 external variables


In [45]:
# Action 1: Given that the definition of the previous column is uncleared, I will drop this column in both train and test data.
train = train.drop(columns=["previous"])
test = test.drop(columns=["previous"])

train.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,unknown,no


In [46]:
# Action 2: Given that the numeric variables have extreme different ranges, I will use Min Max Scaler to normalize all numeric variables.
# Here I normalize the age column with the Min Max Scaler
age_scaler = MinMaxScaler()
age_scaler = age_scaler.fit(train[['age']])
train['age_scaled'] = age_scaler.transform(train[['age']])
test['age_scaled'] = age_scaler.transform(test[['age']])

train = train.drop(columns=['age'])
test = test.drop(columns=['age'])

In [47]:
# Action 3: The job column is a categorical variable. I will convert the job categorical variable into One-Hot encoding dummy variables.
print("Percentage of job categories:")
print("There are totally {} job categories.".format(len(train['job'].value_counts())))
print(round(train['job'].value_counts()*100/train.shape[0],2))

train = pd.get_dummies(train,columns=['job'],drop_first=True)
test =  pd.get_dummies(test,columns=['job'],drop_first=True)
train.head(3)

Percentage of job categories:
There are totally 12 job categories.
blue-collar      21.53
management       20.92
technician       16.80
admin.           11.44
services          9.19
retired           5.01
self-employed     3.49
entrepreneur      3.29
unemployed        2.88
housemaid         2.74
student           2.07
unknown           0.64
Name: job, dtype: float64


,marital,education,default,balance,housing,loan,contact,day,month,duration,...,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,married,tertiary,no,2143,yes,no,unknown,5,may,261,...,0,0,1,0,0,0,0,0,0,0
1,single,secondary,no,29,yes,no,unknown,5,may,151,...,0,0,0,0,0,0,0,1,0,0
2,married,secondary,no,2,yes,yes,unknown,5,may,76,...,1,0,0,0,0,0,0,0,0,0


In [48]:
# Action 4: The marital column is a categorical variable. I will convert the marital categorical variable into One-Hot encoding dummy variables.
print("Percentage of marital categories:")
print("There are totally {} marital categories.".format(len(train['marital'].value_counts())))
print(round(train['marital'].value_counts()*100/train.shape[0],2))

train = pd.get_dummies(train,columns=['marital'],drop_first=True)
test =  pd.get_dummies(test,columns=['marital'],drop_first=True)
train.head(3)

Percentage of marital categories:
There are totally 3 marital categories.
married     60.19
single      28.29
divorced    11.52
Name: marital, dtype: float64


,education,default,balance,housing,loan,contact,day,month,duration,campaign,...,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single
0,tertiary,no,2143,yes,no,unknown,5,may,261,1,...,1,0,0,0,0,0,0,0,1,0
1,secondary,no,29,yes,no,unknown,5,may,151,1,...,0,0,0,0,0,1,0,0,0,1
2,secondary,no,2,yes,yes,unknown,5,may,76,1,...,0,0,0,0,0,0,0,0,1,0


In [49]:
# Action 5: The education column is a categorical variable. I will convert the education categorical variable into One-Hot encoding dummy variables.
print("Percentage of education categories:")
print("There are totally {} education categories.".format(len(train['education'].value_counts())))
print(round(train['education'].value_counts()*100/train.shape[0],2))

train = pd.get_dummies(train,columns=['education'],drop_first=True)
test =  pd.get_dummies(test,columns=['education'],drop_first=True)
train.head(3)

Percentage of education categories:
There are totally 4 education categories.
secondary    51.32
tertiary     29.42
primary      15.15
unknown       4.11
Name: education, dtype: float64


,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,...,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown
0,no,2143,yes,no,unknown,5,may,261,1,-1,...,0,0,0,0,0,1,0,0,1,0
1,no,29,yes,no,unknown,5,may,151,1,-1,...,0,0,1,0,0,0,1,1,0,0
2,no,2,yes,yes,unknown,5,may,76,1,-1,...,0,0,0,0,0,1,0,1,0,0


In [50]:
# Action 6: The default column is a binary variable. I will convert the default binary variable into 1 and 0.
print("Percentage of default categories:")
print("There are totally {} default categories.".format(len(train['default'].value_counts())))
print(round(train['default'].value_counts()*100/train.shape[0],2))

train['default'] = train['default'].replace(['yes','no'],[1,0])
test['default'] = test['default'].replace(['yes','no'],[1,0])
train.head(3)

Percentage of default categories:
There are totally 2 default categories.
no     98.2
yes     1.8
Name: default, dtype: float64


,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,...,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown
0,0,2143,yes,no,unknown,5,may,261,1,-1,...,0,0,0,0,0,1,0,0,1,0
1,0,29,yes,no,unknown,5,may,151,1,-1,...,0,0,1,0,0,0,1,1,0,0
2,0,2,yes,yes,unknown,5,may,76,1,-1,...,0,0,0,0,0,1,0,1,0,0


In [51]:
# Action 7: I normalize the balance column with the Min Max Scaler
balance_scaler = MinMaxScaler()
balance_scaler = balance_scaler.fit(train[['balance']])
train['balance_scaled'] = balance_scaler.transform(train[['balance']])
test['balance_scaled'] = balance_scaler.transform(test[['balance']])

train = train.drop(columns=['balance'])
test = test.drop(columns=['balance'])

In [52]:
# Action 8: The housing column is a binary variable. I will convert the housing binary variable into 1 and 0.
print("Percentage of housing categories:")
print("There are totally {} housing categories.".format(len(train['housing'].value_counts())))
print(round(train['housing'].value_counts()*100/train.shape[0],2))

train['housing'] = train['housing'].replace(['yes','no'],[1,0])
test['housing'] = test['housing'].replace(['yes','no'],[1,0])
train.head(3)

Percentage of housing categories:
There are totally 2 housing categories.
yes    55.58
no     44.42
Name: housing, dtype: float64


,default,housing,loan,contact,day,month,duration,campaign,pdays,poutcome,...,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,balance_scaled
0,0,1,no,unknown,5,may,261,1,-1,unknown,...,0,0,0,0,1,0,0,1,0,0.092259
1,0,1,no,unknown,5,may,151,1,-1,unknown,...,0,1,0,0,0,1,1,0,0,0.073067
2,0,1,yes,unknown,5,may,76,1,-1,unknown,...,0,0,0,0,1,0,1,0,0,0.072822


In [53]:
# Action 9: The loan column is a binary variable. I will convert the loan binary variable into 1 and 0.
print("Percentage of loan categories:")
print("There are totally {} loan categories.".format(len(train['loan'].value_counts())))
print(round(train['loan'].value_counts()*100/train.shape[0],2))

train['loan'] = train['loan'].replace(['yes','no'],[1,0])
test['loan'] = test['loan'].replace(['yes','no'],[1,0])
train.head(3)

Percentage of loan categories:
There are totally 2 loan categories.
no     83.98
yes    16.02
Name: loan, dtype: float64


,default,housing,loan,contact,day,month,duration,campaign,pdays,poutcome,...,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,balance_scaled
0,0,1,0,unknown,5,may,261,1,-1,unknown,...,0,0,0,0,1,0,0,1,0,0.092259
1,0,1,0,unknown,5,may,151,1,-1,unknown,...,0,1,0,0,0,1,1,0,0,0.073067
2,0,1,1,unknown,5,may,76,1,-1,unknown,...,0,0,0,0,1,0,1,0,0,0.072822


In [55]:
# Action 10: The contact column is a categorical variable. I will convert the contact categorical variable into One-Hot encoding dummy variables.
print("Percentage of contact categories:")
print("There are totally {} contact categories.".format(len(train['contact'].value_counts())))
print(round(train['contact'].value_counts()*100/train.shape[0],2))

train = pd.get_dummies(train,columns=['contact'],drop_first=True)
test =  pd.get_dummies(test,columns=['contact'],drop_first=True)
train.head(3)

Percentage of contact categories:
There are totally 3 contact categories.
cellular     64.77
unknown      28.80
telephone     6.43
Name: contact, dtype: float64


,default,housing,loan,day,month,duration,campaign,pdays,poutcome,y,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,balance_scaled,contact_telephone,contact_unknown
0,0,1,0,5,may,261,1,-1,unknown,no,...,0,0,1,0,0,1,0,0.092259,0,1
1,0,1,0,5,may,151,1,-1,unknown,no,...,0,0,0,1,1,0,0,0.073067,0,1
2,0,1,1,5,may,76,1,-1,unknown,no,...,0,0,1,0,1,0,0,0.072822,0,1


In [57]:
# Action 11: The day column is a categorical variable. I believe the day column is less relevant and ranodm.
# In addition, we have the pday column that take the number of days passed by after contact from the previous campaign into account.
# So I drop the day column.
train = train.drop(columns=['day'])
test = test.drop(columns=['day'])
train.head(3)

,default,housing,loan,month,duration,campaign,pdays,poutcome,y,age_scaled,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,balance_scaled,contact_telephone,contact_unknown
0,0,1,0,may,261,1,-1,unknown,no,0.519481,...,0,0,1,0,0,1,0,0.092259,0,1
1,0,1,0,may,151,1,-1,unknown,no,0.337662,...,0,0,0,1,1,0,0,0.073067,0,1
2,0,1,1,may,76,1,-1,unknown,no,0.194805,...,0,0,1,0,1,0,0,0.072822,0,1


In [58]:
# Action 12: The month column is a categorical variable. I believe the month column is less relevant and ranodm.
# In addition, we have the pday column that take the number of days passed by after contact from the previous campaign into account.
# So I drop the month column.
train = train.drop(columns=['month'])
test = test.drop(columns=['month'])
train.head(3)

,default,housing,loan,duration,campaign,pdays,poutcome,y,age_scaled,job_blue-collar,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,balance_scaled,contact_telephone,contact_unknown
0,0,1,0,261,1,-1,unknown,no,0.519481,0,...,0,0,1,0,0,1,0,0.092259,0,1
1,0,1,0,151,1,-1,unknown,no,0.337662,0,...,0,0,0,1,1,0,0,0.073067,0,1
2,0,1,1,76,1,-1,unknown,no,0.194805,0,...,0,0,1,0,1,0,0,0.072822,0,1


In [59]:
# Action 13: I normalize the duration column with the Min Max Scaler
duration_scaler = MinMaxScaler()
duration_scaler = duration_scaler.fit(train[['duration']])
train['duration_scaled'] = duration_scaler.transform(train[['duration']])
test['duration_scaled'] = duration_scaler.transform(test[['duration']])

train = train.drop(columns=['duration'])
test = test.drop(columns=['duration'])

In [62]:
# Action 14: I normalize the campaign column with the Min Max Scaler
campaign_scaler = MinMaxScaler()
campaign_scaler = campaign_scaler.fit(train[['campaign']])
train['campaign_scaled'] = campaign_scaler.transform(train[['campaign']])
test['campaign_scaled'] = campaign_scaler.transform(test[['campaign']])

train = train.drop(columns=['campaign'])
test = test.drop(columns=['campaign'])

In [68]:
# Action 15: I normalize the pdays column with the Min Max Scaler
pdays_scaler = MinMaxScaler()
pdays_scaler = pdays_scaler.fit(train[['pdays']])
train['pdays_scaled'] = pdays_scaler.transform(train[['pdays']])
test['pdays_scaled'] = pdays_scaler.transform(test[['pdays']])

train = train.drop(columns=['pdays'])
test = test.drop(columns=['pdays'])

In [70]:
# Action 16: The poutcome column is a categorical variable. I will convert the poutcome categorical variable into One-Hot encoding dummy variables.
print("Percentage of poutcome categories:")
print("There are totally {} poutcome categories.".format(len(train['poutcome'].value_counts())))
print(round(train['poutcome'].value_counts()*100/train.shape[0],2))

train = pd.get_dummies(train,columns=['poutcome'],drop_first=True)
test =  pd.get_dummies(test,columns=['poutcome'],drop_first=True)
train.head(3)

Percentage of poutcome categories:
There are totally 4 poutcome categories.
unknown    81.75
failure    10.84
other       4.07
success     3.34
Name: poutcome, dtype: float64


,default,housing,loan,y,age_scaled,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,education_unknown,balance_scaled,contact_telephone,contact_unknown,duration_scaled,campaign_scaled,pdays_scaled,poutcome_other,poutcome_success,poutcome_unknown
0,0,1,0,no,0.519481,0,0,0,1,0,...,0,0.092259,0,1,0.053070,0.0,0.0,0,0,1
1,0,1,0,no,0.337662,0,0,0,0,0,...,0,0.073067,0,1,0.030704,0.0,0.0,0,0,1
2,0,1,1,no,0.194805,0,1,0,0,0,...,0,0.072822,0,1,0.015453,0.0,0.0,0,0,1


In [72]:
# Action 17: The y column is a binary variable. I will convert the y binary variable into 1 and 0.
# Apparently the data is extremely imbalanced. 
# To fix the issue, I will adjust the class weights accordingly while fitting the models in the next step.
print("Percentage of y categories:")
print("There are totally {} y categories.".format(len(train['y'].value_counts())))
print(round(train['y'].value_counts()*100/train.shape[0],2))

train['y'] = train['y'].replace(['yes','no'],[1,0])
test['y'] = test['y'].replace(['yes','no'],[1,0])
train.head(3)

Percentage of y categories:
There are totally 2 y categories.
no     88.3
yes    11.7
Name: y, dtype: float64


,default,housing,loan,y,age_scaled,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,education_unknown,balance_scaled,contact_telephone,contact_unknown,duration_scaled,campaign_scaled,pdays_scaled,poutcome_other,poutcome_success,poutcome_unknown
0,0,1,0,0,0.519481,0,0,0,1,0,...,0,0.092259,0,1,0.053070,0.0,0.0,0,0,1
1,0,1,0,0,0.337662,0,0,0,0,0,...,0,0.073067,0,1,0.030704,0.0,0.0,0,0,1
2,0,1,1,0,0.194805,0,1,0,0,0,...,0,0.072822,0,1,0.015453,0.0,0.0,0,0,1


In [75]:
# Save the pre-processed train and test data
train.to_csv("/Users/haochunniu/Desktop/Kaggle Compatition/Bank Term Deposit Predictions NN ML Nested Random Search/data/train_preprocessed.csv",index=False)
test.to_csv("/Users/haochunniu/Desktop/Kaggle Compatition/Bank Term Deposit Predictions NN ML Nested Random Search/data/test_preprocessed.csv",index=False)
